## I am the man who arranges the blocks

In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import sys

In [2]:
jets = [1 if jet == '>' else -1 for jet in Path('tetris.txt').read_text()]

rock_types = ['horizontal', 'cross', 'backl', 'vertical', 'square']

shaft_width = 7

num_types = len(rock_types)

num_jets = len(jets)

In [3]:
# Worst case scenario of rock height is +1, +3, +3, +4, +2 and then cycling

def calculate_max_height(num_rocks):
    heights = [1, 3, 3, 4, 2]
    
    rock_types = len(heights)
    
    total_height = 0
    
    total_height += (num_rocks // rock_types)*sum(heights)
    
    for i in range(num_rocks % rock_types):
        total_height += heights[i]
        
    return total_height

In [4]:
def find_max_rock_height(shaft):
    max_y = np.where(shaft == 1)[0]
    if len(max_y) == 0:
        return 0
    else:
        return np.max(max_y)

In [5]:
# ####

# .#.
# ###
# .#.

# ..#
# ..#
# ###

# #
# #
# #
# #

# ##
# ##

class rock():
    
    # y coordinate of rock is lowest of rock y-coordinates
    # x coordinate of rock is left most coordinate at lowest rock y-coordinate
    def __init__(self, shape, coordinate):
        self.coordinate = coordinate
        self.shape = shape
        
        self.rock_coordinates = self.generate_coordinates(self.coordinate)
        
    def generate_coordinates(self, coordinate):
        
        shape = self.shape
        
        y, x = coordinate
        
        if shape == 'horizontal':
            rock_coordinates = [[y, x], [y, x+1], [y, x+2], [y, x+3]]
        elif shape == 'cross':
            rock_coordinates = [[y, x], [y+1, x-1], [y+1, x], [y+1, x+1], [y+2, x]]
        elif shape == 'backl':
            rock_coordinates = [[y, x], [y, x+1], [y, x+2], [y+1, x+2], [y+2, x+2]]
        elif shape == 'vertical':
            rock_coordinates = [[y, x], [y+1, x], [y+2, x], [y+3, x]]
        elif shape == 'square':
            rock_coordinates = [[y, x], [y, x+1], [y+1, x], [y+1, x+1]]
            
        return rock_coordinates
            
    def move(self, direction, shaft):
        
        y0, x0 = self.coordinate
        
        vy, vx = direction
        
        y1, x1 = [y0 + vy, x0 + vx]
        
        new_coords = self.generate_coordinates([y1, x1])
        
        if check_collision(new_coords, shaft) == False:
            self.coordinate = [y1, x1]
            self.rock_coordinates = self.generate_coordinates(self.coordinate)
            
            return True
        
        else:
            return False
        
def check_collision(coords, shaft):
    height, width = shaft.shape
    
    y_coords = np.array([coord[0] for coord in coords])
    x_coords = np.array([coord[1] for coord in coords])
    
    # Check if within bounds
    if np.all(y_coords >= 1) and np.all(y_coords < height) and np.all(x_coords >= 0) and np.all(x_coords < width):
        # Check for collision
        return np.any(np.array([shaft[tuple(coord)] for coord in coords]) == 1)
    
    else:
        return True

In [6]:
def spawn_rock(shape, height):
    if shape in ['horizontal', 'backl', 'vertical', 'square']:
        return rock(shape, [height, 2])
    elif shape == 'cross':
        return rock(shape, [height, 3])

In [7]:
def add_rock(shaft, rock):
    for coord in rock.rock_coordinates:
        shaft[tuple(coord)] = 1
        
    return shaft

In [35]:
num_rocks = 2022

shaft_width = 7

shaft = np.zeros((calculate_max_height(num_rocks)+8, shaft_width))

rock_types = ['horizontal', 'cross', 'backl', 'vertical', 'square']

num_types = len(rock_types)

num_jets = len(jets)

jet_counter = 0

for i in range(num_rocks):
    current_rock_type = rock_types[i % num_types]
    
    current_max_height = find_max_rock_height(shaft)
    
    current_rock = spawn_rock(current_rock_type, current_max_height + 4)
    
    current_rock_moving = True
    
    while current_rock_moving:
        current_rock.move([0, jets[jet_counter % num_jets]], shaft)
        jet_counter += 1
        jet_counter %= num_jets
        
        fall = current_rock.move([-1, 0], shaft)
        
        if not fall:
            current_rock_moving = False
            
    shaft = add_rock(shaft, current_rock)
    
print(find_max_rock_height(shaft))

3184


## Part 2: Pointless work for pointless pay
## This is one game I shall not play

In [17]:
def build(start_rock, start_jet, rocks, shaft, print_cycle = True):
    
    shaft = np.copy(shaft)
    
    rock_counter = 0
    jet_counter = start_jet
    
    blocking_heights = [i for i in range(0, find_max_rock_height(shaft)) if np.all(shaft[i] == 1)]
    
    for i in range(start_rock, start_rock+rocks):   
        
        current_rock_type = rock_types[i % num_types]

        current_max_height = find_max_rock_height(shaft)

        current_rock = spawn_rock(current_rock_type, current_max_height + 4)

        current_rock_moving = True

        while current_rock_moving:
            current_rock.move([0, jets[jet_counter % num_jets]], shaft)
            
            # Try to detect cycle:
            if jet_counter == 0 and print_cycle:
                print(current_rock_type, rock_counter+1)
            
            jet_counter += 1
            jet_counter %= num_jets

            fall = current_rock.move([-1, 0], shaft)

            if not fall:
                current_rock_moving = False

        shaft = add_rock(shaft, current_rock)
        
        rock_counter += 1
        
    return shaft
        
    

In [15]:
shaft = build(0, 0, 10000, np.zeros((calculate_max_height(10000)+8, shaft_width)))

horizontal 1
backl 1748
backl 3493
backl 5238
backl 6983
backl 8728


In [18]:
# Looks like cycle of 1745 rocks

heights = [find_max_rock_height(shaft) for shaft in 
             [build(0, 0, rocks, np.zeros((calculate_max_height(rocks)+8, shaft_width)), False) 
              for rocks in range(1748, 1748+1745*10+1, 1745)]]

In [19]:
heights

[2775, 5527, 8279, 11031, 13783, 16535, 19287, 22039, 24791, 27543, 30295]

In [20]:
[heights[i] - heights[i-1] for i in range(1, len(heights))]

[2752, 2752, 2752, 2752, 2752, 2752, 2752, 2752, 2752, 2752]

In [31]:
cycle_start = 1748
cycle_length = 1745

initial_height = 2775
height_per_cycle = 2752

cycles = (1000000000000 - cycle_start)//cycle_length
remainder = (1000000000000 - cycle_start)%cycle_length

remainder_height = find_max_rock_height(build(0, 0, 1748+remainder, 
                                              np.zeros((calculate_max_height(1748+remainder)+8, shaft_width)), False))
remainder_height -= initial_height

total_height = initial_height + cycles*height_per_cycle + remainder_height

print(total_height)

1577077363915
